In [1]:
# %pip install -q langchain langchain-openai langchain-community "pydantic>=2"

In [2]:
from typing import List
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import CommaSeparatedListOutputParser

def build_llm(model_name: str = "gpt-4o-mini", temperature: float = 0.2):
    return ChatOpenAI(model=model_name, temperature=temperature)

def recommend_korea_items(topic: str, llm=None) -> List[str]:
    """
    사용자 관심 분야(topic)를 받아,
    한국의 유명한 장소/활동 5개를 콤마 구분 리스트로 파싱해서 반환.
    """
    llm = llm or build_llm()
    parser = CommaSeparatedListOutputParser()
    fmt = parser.get_format_instructions()

    prompt = PromptTemplate(
        template=(
            """
            너의 임무는 사용자가 입력한 관심 분야와 관련된 **한국의 유명한 장소 또는 활동** 5가지를 제안하는 것이다.
            - 반드시 한국과 연관된 항목만 제시할 것
            - 중복 없이 5개 정확히
            - {format_instructions}

            관심 분야: {topic}
            """
        ),
        input_variables=["topic"],
        partial_variables={"format_instructions": fmt},
    )

    chain = prompt | llm | parser
    result = chain.invoke({"topic": topic})

    if isinstance(result, list):
        return result[:5]
    return result


In [3]:
user_topic = input('관심 분야를 입력하세요 (예: 음식, 스포츠, 영화 등): ').strip()
if not user_topic:
    user_topic = "음식" 

items = recommend_korea_items(user_topic)
print(items) 

['김치전', '비빔밥', '떡볶이', '한정식', '삼겹살']
